# Milestone 1 : Regression - Predicting seismic collapse capacity

#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import metrics

## 1.Data

### 1.1 Import

In [ ]:
df_train = pd.read_csv("Data/train_set.csv")
df_test = pd.read_csv("Data/test_set.csv")
df_val = pd.read_csv("Data")

In [ ]:
df = {}
X = {}
X_reduced = {} #using only last columns, including sa_ratio
y = {}
col_reduced = ['sa_avg','da5_75', 'da5_95', 'fiv3', 'sa_ratio']

for t in ['train', 'test', 'val'] :
    df[t] = pd.read_csv(f"Data/{t}_set.csv")
    X[t] = df[t][df[t].columns.difference(['sat1_col'])].to_numpy()
    X_reduced[t] = df[t][col_reduced].to_numpy()
    if t != 'test':
        y[t] = df[t]['sat1_col'].to_numpy()

print(X_reduced['train'].shape)

#df_train['1.3']/df_train['sa_avg']/df_train['sa_ratio']

(12646, 5)


### 1.2 Normalisation

In [ ]:
mean = X['train'].mean(axis=0)
std = X['train'].std(axis=0)
def Normalise(X):
    return (X-mean)/std

for t in X.keys() :
    X[t] = Normalise(X[t])

mean = X_reduced['train'].mean(axis=0)
std = X_reduced['train'].std(axis=0)

for t in X.keys() :
    X_reduced[t] = Normalise(X_reduced[t])


### 1.3 Import en Pytorch

https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader

In [ ]:
dataloader = {"Normal" : {}, "Reduced" : {}}

for t in y.keys():
    tensor_x = torch.Tensor(X[t])
    tensor_x_red = torch.Tensor(X_reduced[t])
    tensor_y = torch.Tensor(y[t])
    dataset_x = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    dataset_x_red = torch.utils.data.TensorDataset(tensor_x_red,tensor_y)
    dataloader['Normal'][t] = torch.utils.data.DataLoader(dataset_x, batch_size = 100)
    dataloader['Reduced'][t] = torch.utils.data.DataLoader(dataset_x_red, batch_size = 100)

## 2. Models

In [ ]:
models = {"Normal" : {}, "Reduced" : {}}

### 2.1 OneLayerNet

In [ ]:
class OneLayerNet(nn.Module):
    """1-Layer MNIST classifier"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        return x
  
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

models['Normal']['1Layer'] = OneLayerNet(X['train'].shape[1]) 
models['Reduced']['1Layer'] = OneLayerNet(X_reduced['train'].shape[1])

## 3. Training

### 3.1 Loss function and optimiser

In [ ]:
loss_fn = nn.MSELoss()

### 3.2 Training

In [ ]:
def train(model: torch.nn.Module, train_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, epochs: int):
    
    # Initialize metrics for loss and accuracy
    #loss_metric = metrics.LossMetric()
    losses = []
    
    # Sets the module in training mode (doesn't have any effect here, but good habit to take)
    model.train()
    
    for epoch in range(1, epochs + 1):
        
        # Progress bar set-up
        #pbar = tqdm(total=len(train_loader), leave=True)
        #pbar.set_description(f"Epoch {epoch}")
        
        # Iterate through data
        for data, target in train_loader:
            
            ### START CODE HERE ###
            
            # Zero-out the gradients
            optimizer.zero_grad()
            
            # Forward pass
            out = model(data)
            
            # Compute loss
            loss = loss_fn(out, target.view(-1,1))
            
            # Backward pass
            loss.backward()
            
            # Optimizer stepdef closure():
            def closure():
                optimizer.zero_grad()
                output = model(data)
                loss = loss_fn(output, target)
                loss.backward()
                return loss
            optimizer.step(closure)
            
            ### END CODE HERE ###
            
            # Update metrics & progress bar
            #loss_metric.update(loss.item(), data.shape[0])
            #pbar.update()
            losses.append(loss)
            
        # End of epoch, show loss and acc
        #pbar.set_postfix_str(f"Train loss: {loss_metric.compute():.3f} | Train acc: {acc_metric.compute() * 100:.2f}%")
        #print(f"Train loss: {loss_metric.compute():.3f}")
        #loss_metric.reset()

        print(epoch, losses)

In [ ]:
t = 'Normal'
choice = '1Layer'
model = models[t][choice]
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum= 0.9)
train_loader = dataloader[t]['train']
train(model, train_loader, loss_fn, optimizer, epochs=10)

/shared-libs/python3.7/py/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/shared-libs/python3.7/py/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
1 [tensor(3.4545, grad_fn=<MseLossBackward>), tensor(2.6077, grad_fn=<MseLossBackward>), tensor(2.5055, grad_fn=<MseLossBackward>), tensor(2.0017, grad_fn=<MseLossBackward>), tensor(1.7856, grad_fn=<MseLossBackward>), tensor(1.4205, grad_fn=<MseLossBackward>), tensor(3.1397, grad

In [ ]:
for data, target in dataloader[t]['val']:
    print(*zip(model(data), target.view(-1,1)))

(tensor([-4.4305e+23], grad_fn=<UnbindBackward>), tensor([1.6500])) (tensor([-4.6839e+23], grad_fn=<UnbindBackward>), tensor([1.2900])) (tensor([2.2938e+24], grad_fn=<UnbindBackward>), tensor([1.4000])) (tensor([-6.1231e+23], grad_fn=<UnbindBackward>), tensor([1.9000])) (tensor([5.1665e+23], grad_fn=<UnbindBackward>), tensor([2.3400])) (tensor([-3.6774e+23], grad_fn=<UnbindBackward>), tensor([1.6000])) (tensor([-5.2914e+23], grad_fn=<UnbindBackward>), tensor([1.0300])) (tensor([-1.5910e+23], grad_fn=<UnbindBackward>), tensor([1.3000])) (tensor([-5.6201e+23], grad_fn=<UnbindBackward>), tensor([0.6900])) (tensor([1.3512e+24], grad_fn=<UnbindBackward>), tensor([0.9500])) (tensor([1.8418e+23], grad_fn=<UnbindBackward>), tensor([1.3100])) (tensor([6.5960e+23], grad_fn=<UnbindBackward>), tensor([0.8800])) (tensor([-8.9495e+22], grad_fn=<UnbindBackward>), tensor([0.6500])) (tensor([4.8636e+23], grad_fn=<UnbindBackward>), tensor([1.5000])) (tensor([-6.3187e+23], grad_fn=<UnbindBackward>), tens

In [ ]:
print(model)

OneLayerNet(
  (fc1): Linear(in_features=110, out_features=1, bias=True)
)


In [ ]:
loss_fn(torch.tensor([[0.0,1]]), torch.tensor([[0.0,0]]))

tensor(0.5000)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ad5b9508-4536-4617-b060-a55c942dfcf5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>